# o'reillyのツノガレイ強化学習の本

## 深層学習で強化学習

## [目次](TableOfContents.ipynb)
- [環境準備](#環境準備)
  - [インストール](#インストール)
  - [インポート](#インポート)
  - [共通関数](#共通関数)
- [DeZero](#DeZero)
  - [matmul](#matmul)
  - [微分計算](#微分計算)
  - [回帰問題](#回帰問題)
    - [線形モデルの回帰](#線形モデルの回帰)
    - [非線形モデルの回帰](#非線形モデルの回帰)
- [DQN未満](#DQN未満)
  - [エージェントの実装1](#エージェントの実装1)
  - [エージェントの実行1](#エージェントの実行1)
    - [DQN未満学習](#DQN未満学習)
    - [学習履歴の表示](#学習履歴の表示)
    - [行動価値関数の可視化](#行動価値関数の可視化)
- [DQN](#DQN)
  - [倒立振子問題](#倒立振子問題)
  - [経験再生（リプレイ・バッファ）](#経験再生（リプレイ・バッファ）)
    - [クラス定義](#クラス定義)
    - [クラス試用](#クラス試用)
      - [記録](#記録)
      - [再生](#再生)
  - [DQNの実装](#DQNの実装)
    - [エージェントの実装2](#エージェントの実装2)
    - [エージェントの実行2](#エージェントの実行2)
    - [実行結果の描画](#実行結果の描画)

## 参考
- https://github.com/oreilly-japan/deep-learning-from-scratch-4/tree/master/ch01
- [強化学習（Reinforcement Learning） - .NET 開発基盤部会 Wiki](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E5%BC%B7%E5%8C%96%E5%AD%A6%E7%BF%92%EF%BC%88Reinforcement%20Learning%EF%BC%89)

## 環境準備

### インストール

In [ ]:
!pip install numpy
!pip install tabulate
!pip install matplotlib
!pip install dezero
!pip install dezerogym
!pip install gym[classic_control]

### インポート

In [ ]:
import numpy as np
import copy
import random
from collections import deque
from tabulate import tabulate
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# 廃止となったエイリアスを書く
np.object = object
np.bool = bool
np.int = int
np.float = float
np.typeDict = {k: v for k, v in np.sctypeDict.items() if isinstance(v, type)}

from dezero import Variable
from dezero import Model
from dezero import optimizers
import dezero.functions as F
import dezero.layers as L

from dezerogym.gridworld import GridWorld
import gym # OpenAI Gym

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定

### 共通関数

#### 状態のone_hot化

In [ ]:
def one_hot(state):
    HEIGHT, WIDTH = 3, 4 # 3*4 グリッドワールド
    vec = np.zeros(HEIGHT * WIDTH, dtype=np.float32)
    y, x = state
    idx = WIDTH * y + x
    vec[idx] = 1.0
    return vec[np.newaxis, :]

#### 行動価値関数のDNN版

##### QNet1
入力x → 隠れ層100 → 出力層4（アクション数と同じ）

In [ ]:
class QNet1(Model):
    def __init__(self):
        super().__init__()
        self.l1 = L.Linear(100)  # hidden_size
        self.l2 = L.Linear(4)  # action_size

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = self.l2(x)
        return x

##### QNet2
入力x → 隠れ層 128 128 → 出力層4（アクション数と同じ）

In [ ]:
class QNet2(Model):
    def __init__(self, action_size):
        super().__init__()
        self.l1 = L.Linear(128)
        self.l2 = L.Linear(128)
        self.l3 = L.Linear(action_size)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = self.l3(x)
        return x

#### OpenAI Gym 表示領域定義

In [ ]:
def define_area (state):
    # ラベル
    state_text = f'cart position={state[0]:5.2f}, '
    state_text += f'cart velocity={state[1]:6.3f}\n'
    state_text += f'pole angle   ={state[2]:5.2f}, '
    state_text += f'pole velocity={state[3]:6.3f}'

    # 領域定義
    fig = plt.figure(figsize=(9, 7), facecolor='white')
    plt.suptitle('Cart Pole', fontsize=20)
    plt.xticks(ticks=[])
    plt.yticks(ticks=[])
    plt.title(state_text, loc='left')
    
    return fig

### DeZero
ゼロから作るDeep Learning 3 ―フレームワーク編で開発されたDeep Learningフレームワーク

#### matmul
ベクトル内積、行列ドット積ができる。

In [ ]:
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
a, b = Variable(a), Variable(b)  # Optional
c = F.matmul(a, b)
print(c)

# Matrix product
a = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6], [7, 8]])
c = F.matmul(a, b)
print(c)

#### 微分計算
rosenbrock関数を微分して勾配降下していく。

In [ ]:
# rosenbrock関数の重みはx0, x1
def rosenbrock(x0, x1):
    y = 100 * (x1 - x0 ** 2) ** 2 + (x0 - 1) ** 2
    return y

x0 = Variable(np.array(0.0))
x1 = Variable(np.array(2.0))

lr = 0.001
iters = 10000

for i in range(iters):
    
    # 順伝播
    y = rosenbrock(x0, x1)

    # 勾配クリア
    x0.cleargrad()
    x1.cleargrad()
    
    # 逆伝播で勾配計算
    y.backward() # y = rosenbrockの微分

    # 最適化
    x0.data -= lr * x0.grad.data
    x1.data -= lr * x1.grad.data

print(x0, x1)

#### 回帰問題

##### 線形モデルの回帰

In [ ]:
# トイ・データセット
np.random.seed(0)
x = np.random.rand(100, 1)
y = 5 + 2 * x + np.random.rand(100, 1)
x, y = Variable(x), Variable(y)  # 省略可能

W = Variable(np.zeros((1, 1)))
b = Variable(np.zeros(1))

def predict(x):
    y = F.matmul(x, W) + b
    return y

def mean_squared_error(x0, x1):
    diff = x0 - x1
    return F.sum(diff ** 2) / len(diff)

lr = 0.1
iters = 100

for i in range(iters):
    y_pred = predict(x)
    loss = mean_squared_error(y, y_pred)

    W.cleargrad()
    b.cleargrad()
    loss.backward() # loss = 損失関数()の微分

    # 重みの更新
    W.data -= lr * W.grad.data
    b.data -= lr * b.grad.data

    # 損失の表示
    if i % 10 == 0:
        print(loss.data)

print('====')
print('W =', W.data)
print('b =', b.data)

# 回帰直線のPlot
plt.scatter(x.data, y.data, s=10)
plt.xlabel('x')
plt.ylabel('y')
t = np.arange(0, 1, .01)[:, np.newaxis]
y_pred = predict(t)
plt.plot(t, y_pred.data, color='r')
plt.show()

##### 非線形モデルの回帰

In [ ]:
# Dataset
np.random.seed(0)
x = np.random.rand(100, 1)
y = np.sin(2 * np.pi * x) + np.random.rand(100, 1)

lr = 0.2
iters = 10000

class TwoLayerNet(Model):
    def __init__(self, hidden_size, out_size):
        super().__init__()
        self.l1 = L.Linear(hidden_size)
        self.l2 = L.Linear(out_size)

    def forward(self, x):
        y = F.sigmoid(self.l1(x))
        y = self.l2(y)
        return y

model = TwoLayerNet(10, 1)
optimizer = optimizers.SGD(lr)
optimizer.setup(model)

for i in range(iters):
    y_pred = model(x)
    loss = F.mean_squared_error(y, y_pred)

    model.cleargrads()
    loss.backward() # loss = 損失関数()の微分

    # 重みの更新
    optimizer.update()
    
    # 損失の表示
    if i % 1000 == 0:
        print(loss.data)

# 回帰曲線のPlot
plt.scatter(x, y, s=10)
plt.xlabel('x')
plt.ylabel('y')
t = np.arange(0, 1, .01)[:, np.newaxis]
y_pred = model(t)
plt.plot(t, y_pred.data, color='r')
plt.show()

### DQN未満
Q-Net（行動価値関数のDNN版）を適用したダケのQ学習は、まだDQN未満

#### エージェントの実装1
ココでは（方策オンの）Q学習をベースに実装しているが、コレは、  
後に、オンライン学習からミニバッチ学習に変更するために、  
経験再生を実装する際、方策オフ型である必要があるため。

In [ ]:
class QLearningAgent:
    def __init__(self):
        self.gamma = 0.9
        self.lr = 0.01
        self.epsilon = 0.1
        self.action_size = 4

        # 行動価値関数をNN化したQNet1を利用
        self.qnet = QNet1()
        self.optimizer = optimizers.SGD(self.lr)
        self.optimizer.setup(self.qnet)

    # ε-greedyで行動を選択
    def get_action(self, state_vec):
        if np.random.rand() < self.epsilon:
            return np.random.choice(self.action_size)
        else:
            qs = self.qnet(state_vec)
            return qs.data.argmax()

    # 方策改善（ステップ毎）
    def update(self, state, action, reward, next_state, done):
        
        # next_q_maxを計算
        if done:
            next_q_max = np.zeros(1)  # [0.]
        else:
            next_qs = self.qnet(next_state)
            next_q_max = next_qs.max(axis=1)
            next_q_max.unchain() # 勾配の計算から除外

        # ココがQ学習
        target = reward + self.gamma * next_q_max
        
        # ベルマン的先読みで精度を上げるのではなく、
        # 正解ラベルとして方策NNを学習させて行動価値関数の精度を上げつつ、方策更新。
        
        # ・推定（順伝播）
        qs = self.qnet(state)
        q = qs[:, action]
        
        # ・学習（逆伝播）
        loss = F.mean_squared_error(target, q) # 損失関数で損失計算
        self.qnet.cleargrads() # 勾配を初期化
        loss.backward() # loss = 損失関数()の微分（誤差逆伝播）
        self.optimizer.update() # オンライン学習でパラメタ更新

        return loss.data

#### エージェントの実行1
グリッド・ワールド上で

##### DQN未満学習

In [ ]:
env = GridWorld()
agent = QLearningAgent()

# 1000エピソード
episodes = 1000
loss_history = []
for episode in range(episodes):
    
    # 環境の初期化
    state = env.reset()
    # 状態のone_hot化
    state = one_hot(state)
    # その他の初期化
    total_loss, cnt = 0, 0
    done = False

    # done=tureまでが1エピソード
    while not done:
        cnt += 1
        
        # ActionでStep
        action = agent.get_action(state)
        next_state, reward, done = env.step(action)
        next_state = one_hot(next_state) # 状態のone_hot化

        # 都度学習 ≒ オンライン学習
        loss = agent.update(state, action, reward, next_state, done)
        total_loss += loss # 履歴用
        state = next_state # 次処理用

    # 履歴用
    average_loss = total_loss / cnt
    loss_history.append(average_loss)

##### 学習履歴の表示
深層学習の損失の履歴を表示。

In [ ]:
plt.xlabel('episode')
plt.ylabel('loss')
plt.plot(range(len(loss_history)), loss_history)
plt.show()

##### 行動価値関数の可視化
行動価値関数をNN化したQNetの可視化

In [ ]:
# visualize
Q = {}
for state in env.states():
    for action in env.action_space:
        q = agent.qnet(one_hot(state))[:, action]
        Q[state, action] = float(q.data)
env.render_q(Q)

### DQN

#### 倒立振子問題
- アニメーション表示できないので...
- PoleAngle が±0.23前後になると終了する。

In [ ]:
# 環境のインスタンスを作成
env = gym.make('CartPole-v1', render_mode='rgb_array')

# 状態を初期化
state, info = env.reset()

fig = define_area(state)
list=[]
row = 10
col = 10
num = 0
#ims = []

done = False
#while not done:
for num in range(100):

    # 描画データ
    rgb_data = env.render()
    
    # アクション（ランダム）
    action = np.random.choice([0, 1])
    next_state, reward, done, info, _ = env.step(action)
    
    # 表示データ蓄積
    list.append([str(action), str(reward), str(next_state[0]), str(next_state[1]), str(next_state[2]), str(next_state[3]), str(done), str(info), str(_)])
    num += 1
    plt.subplot(row, col, num)
    plt.imshow(rgb_data)
    #ims.append([plt.imshow(rgb_data, animated=True)])
    
# データ表示
print(tabulate(list, headers=["action", "reward", "CartPosition", "CartVelocity", "PoleAngle", "PoleVelocityAtTip", "done", "info", "_"], tablefmt='grid'))
#ani = animation.ArtistAnimation(fig, ims, interval=20, blit=True, repeat_delay=0)
#ani.save('anim.gif', writer="imagemagick")
#ani.save('anim.mp4', writer="ffmpeg")
plt.show()

#### 経験再生（リプレイ・バッファ）
- オンライン学習からミニバッチ学習に変更するのが目的。
- 行動価値関数が時系列の影響を受けないケースでは時系列的な偏りが無い方が良い。

##### クラス定義

In [ ]:
class ReplayBuffer:

    def __init__(self, buffer_size, batch_size):
        self.buffer = deque(maxlen=buffer_size)
        self.batch_size = batch_size
    
    def add(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))
    
    def __len__(self):
        return len(self.buffer)
    
    def get_batch(self):
        # サンプルデータをランダムに抽出
        data = random.sample(self.buffer, self.batch_size)
        
        # 状態・行動・報酬・次の状態・終了フラグを抽出
        state = np.stack([x[0] for x in data])
        action = np.array([x[1] for x in data])
        reward = np.array([x[2] for x in data])
        next_state = np.stack([x[3] for x in data])
        done = np.array([x[4] for x in data]).astype(np.int32) # 因子型から整数型に変換
        return state, action, reward, next_state, done

##### クラス試用

###### 記録

In [ ]:
# インスタンスを作成
env = gym.make('CartPole-v1')

# サンプルデータの保存数を指定
buffer_size = 100
# ミニバッチのデータ数を指定
batch_size = 32

replay_buffer = ReplayBuffer(buffer_size, batch_size)

# エピソード数を指定
episodes = 10

# 繰り返しシミュレーション
for episode in range(episodes):
    
    # 状態を初期化
    t = 0 # 時刻
    done = False
    state, info = env.reset()

    # done=tureまでが1エピソード
    while not done:
        t += 1
        
        # ActionでStep
        action = np.random.choice([0, 1])
        next_state, reward, done, truncated, info = env.step(action)
        
        # サンプルデータを保存
        replay_buffer.add(state, action, reward, next_state, done)
        
    # 当該エピソードのサンプルデータ数を表示
    # dequeで実装されているので、サイズ以上に増えない。
    print(
        'episode ' + str(episode+1) + 
        ', T=' + str(t) + 
        ', buffer size:' + str(len(replay_buffer))
    )

###### 再生

In [ ]:
# 現在の状態・行動・報酬・次の状態・終了フラグのミニバッチデータを取得
state, action, reward, next_state, done = replay_buffer.get_batch()
print("<state>")
print(state[:5].round(3))
print(state.shape)
print("\n<action>")
print(action[:5])
print(action.shape)
print("\n<reward>")
print(reward[:5])
print(reward.shape)
print("\n<next_state>")
print(next_state[:5].round(3))
print(next_state.shape)
print("\n<done>")
print(done[:5])
print(done.shape)

#### DQNの実装

##### エージェントの実装2

In [ ]:
# DQNのエージェントの実装
class DQNAgent:
    # 初期化メソッドの定義
    def __init__(self):
        # ハイパーパラメタを指定
        self.gamma = 0.98 # 収益の計算用の割引率
        self.lr = 0.0005 # 勾配降下法用の学習率
        self.epsilon = 0.1 # ランダムに行動する確率
        self.buffer_size = 10000 # サンプルデータの保存数
        self.batch_size = 32 # ミニバッチのデータ数
        self.action_size = 2 # 行動の種類数
        
        # インスタンスを作成
        
        # リプレイ・バッファ
        self.replay_buffer = ReplayBuffer(self.buffer_size, self.batch_size)
        
        # リプレイ・バッファを使う場合は方策オフ
        self.qnet_target = QNet2(self.action_size) # 価値推定用の方策
        self.qnet = QNet2(self.action_size) # データ収集用の方策
        self.optimizer = optimizers.Adam(self.lr) # 最適化手法
        self.optimizer.setup(self.qnet) # モデルを設定
    
    # 方策NNのパラメタ同期
    def sync_qnet(self):
        self.qnet_target = copy.deepcopy(self.qnet)
    
    # データ収集用の方策からε-greedy法により行動を選択
    def get_action(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.choice(self.action_size)
        else:
            state = state[np.newaxis, :] # バッチ対応
            qs = self.qnet(state)
            return qs.data.argmax()
    
    # 方策反復（ステップ毎）
    def update(self, state, action, reward, next_state, done):
        
        # ステップの履歴を追加（dequeで実装）
        self.replay_buffer.add(state, action, reward, next_state, done)
        
        # ミニバッチ学習なので学習にはバッチサイズ分のステップが必要
        if len(self.replay_buffer) < self.batch_size: return None
        
        # バッチサイズ分の学習データを取り出し
        state, action, reward, next_state, done = self.replay_buffer.get_batch()
        
        # next_q_maxを計算（価値推定用の方策を用いる）
        next_qs = self.qnet_target(next_state) # 全ての行動
        next_q_max = next_qs.max(axis=1) # 最大値の行動
        next_q_max.unchain() # 勾配の計算から除外
                
        # ココがQ学習
        # target = reward + self.gamma * next_q_max
        target = reward + (1 - done) * self.gamma * next_q_max
        
        # ベルマン的先読みで精度を上げるのではなく、
        # 正解ラベルとしてデータ収集用の方策NNを学習させて行動価値関数の精度を上げつつ、方策更新。
        
        # ・推定（順伝播）
        qs = self.qnet(state)
        q = qs[np.arange(self.batch_size), action]
        
        # ・学習（逆伝播）
        loss = F.mean_squared_error(q, target) # 損失関数で損失計算
        self.qnet.cleargrads() # 勾配を初期化
        loss.backward() # loss = 損失関数()の微分（誤差逆伝播）
        self.optimizer.update() # ミニバッチ学習でパラメタ更新
        
        # 価値推定用の方策NNは、数エピソード間隔で外から同期更新
        
        return loss.data

##### エージェントの実行2

In [ ]:
env = gym.make('CartPole-v1')
agent = DQNAgent()

# 方策NNのパラメタ同期タイミング
sync_interval = 20
# 推移の確認用のリストを初期化
trace_loss = []
trace_reward = []

# 300エピソード
episodes = 300
for episode in range(episodes):
    
    # 環境の初期化
    state, info = env.reset()
    # その他の初期化
    t = 0 # 時刻
    total_loss = 0.0
    total_reward = 0.0
    done = False
    
    # done=tureまでが1エピソード
    while not done:
        t += 1
        
        # ActionでStep
        action = agent.get_action(state)
        next_state, reward, done, truncated, info = env.step(action)
        
        # 都度学習（ミニバッチ化されている
        loss = agent.update(state, action, reward, next_state, done)
        
        # 状態を更新
        state = next_state
        
        # 合計損失を計算（実際に学習した場合）
        if loss != None: total_loss += loss
        # 合計報酬を計算
        total_reward += reward
        
    # 方策NNのパラメタ同期
    if episode % sync_interval == 0: agent.sync_qnet()
        
    # 平均損失・合計報酬を記録
    trace_loss.append(total_loss / t)
    trace_reward.append(total_reward)
    
    # 一定回数ごとに結果を表示
    if (episode+1) % 20 == 0:
        print(
            'episode ' + str(episode+1) + 
            ', T=' + str(t) + 
            ', average loss=' + str(np.round(total_loss/t, 3)) + 
            ', total reward=' + str(total_reward)
        )

##### 実行結果の描画

In [ ]:
# 最適化手法名を取得
optm_name = agent.optimizer.__class__.__name__

# 学習率名を指定
lr_name = 'alpha'

# 学習率を取得
lr = getattr(agent.optimizer, lr_name)

In [ ]:
# 平均損失の推移を作図
plt.figure(figsize=(8, 6), facecolor='white')
plt.plot(np.arange(1, episodes+1), trace_loss)
plt.xlabel('episode')
plt.ylabel('average loss')
plt.suptitle('DQN', fontsize=20)
plt.title(optm_name+': '+lr_name+'='+str(lr), loc='left')
plt.grid()
plt.show()

In [ ]:
# 総報酬の推移を作図
plt.figure(figsize=(8, 6), facecolor='white')
plt.plot(np.arange(1, episodes+1), trace_reward)
plt.xlabel('episode')
plt.ylabel('total reward')
plt.suptitle('DQN', fontsize=20)
plt.title(optm_name+': '+lr_name+'='+str(lr), loc='left')
plt.grid()
plt.show()